# Run a test of hypedsearch with generated data
The following steps describe how the test works
1. Load a fasta database
2. Generate
    1. Hybrid proteins
    2. Peptides
    3. Hybrid peptides from the hybrid proteins
3. Generate spectra for all the peptides created
4. Run hypedsearch with the .fasta file (no hybrid proteins included) and the spectra files
5. Load the summary.json file created
6. Determine what number of alignments were correct

## 1. Load fasta database

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.file_io import fasta

fasta_file = '../data/databases/100prots.fasta'
database = fasta.read(fasta_file, True)

database = {x['name']: x for x in database}

## 2.  Generate the peptides, hybrid proteins and peptides

In [2]:
from sequence_generation import proteins, peptides
test_directory = '../data/testing_output/'

num_hybs = 5
min_length= 5
max_length = 35
num_peptides = 1000
min_cont = 3 #min contribution for each side of a hybrid

# make hybrid proteins
hyb_prots = proteins.generate_hybrids([x for _, x in database.items()], num_hybs, min_contribution=max_length)
# create peptides
non_hybrid_peps = peptides.gen_peptides([x for _, x in database.items()], num_peptides, min_length=min_length, max_length=max_length, digest='random', dist='beta')
# create hybrid peptides
hyb_peps = peptides.gen_peptides(hyb_prots, num_hybs, min_length=min_length, max_length=max_length, digest='random', min_contribution=min_cont, hybrid_list=True)

all_proteins_raw = [x for _,x in database.items()] + hyb_prots
all_peptides_raw = non_hybrid_peps + hyb_peps

peptides = {}
for i, pep in enumerate(all_peptides_raw):
    peptides[i] = pep
    peptides[i]['scan_no'] = i

Generating hybrid protein 4/5[80%]
Finished generating hybrid proteins


## 2.1 Save this info so that I can analyze it later from Neo-Fusion

In [3]:
import json
experiment_info_file_name = 'experiment_info.json'

exp = {'database': fasta_file, 'peptides': peptides}
with open(test_directory + experiment_info_file_name, 'w') as o:
    json.dump(exp, o)


## 2.2 Load data if available instead of creating it

In [4]:
import json

expfile = '../data/testing_output/experiment_info.json'
exp = json.load(open(expfile, 'r'))
peptides = exp['peptides']

## 3. Generate spectra

In [4]:
from src.spectra import gen_spectra
from src.utils import utils
from sequence_generation import write_spectra

utils.make_dir(test_directory)

spectra = []
sorted_keys = [int(c) for c in peptides.keys()]
sorted_keys.sort()
for k in sorted_keys:
    pep = peptides[k]
    cont = gen_spectra.gen_spectrum(pep['sequence'])
    spec = cont['spectrum']
    pm = cont['precursor_mass']
    spectra.append({'spectrum': spec, 'precursor_mass': pm})
write_spectra.write_mzml('testSpectraFile', spectra, output_dir=test_directory)


Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


'../data/testing_output/testSpectraFile.mzML'

## 4. Run hypedsearch

In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import runner
from time import time

test_directory = '../data/testing_output/'
fasta_file = '../data/databases/100prots.fasta'

args = {
    'spectra_folder': test_directory,
    'database_file': fasta_file,
    'output_dir': test_directory,
    'min_peptide_len': 3,
    'max_peptide_len': 35,
}
st = time()
runner.run(args)
print('\nTotal runtime: {} seconds'.format(time() - st))

Loading database...
Done. Indexing database...
7487 unique kmers
Done.
Number of 3-mers found in the database: 7487
Analyzing spectra file 0/1[0%]

Analyzing spectrum 1244/1245[99%]
Total runtime: 1110.4736912250519 seconds


## 5. Load the summary json

In [6]:
import json
test_directory = '../data/testing_output/'

summary = json.load(open(test_directory + 'summary.json', 'r'))
print(test_directory)

../data/testing_output/


#### summary format
Each spectra entry has 3 candidates with the form
```python
{
    "starting_position": 64,
    "ending_position": 73,
    "length": 10,
    "b_score": 4.0,
    "y_score": 4.0,
    "confidence": 100.0,
    "protein_name": "insulin",
    "spectrum": [],
    "sequence": "QVELGGGPGA"
}
```

## 6. Determine which number of alignments were correct
This needs to be broken down into hybrid and non hybrid peptides to get some stats on how well its doing

In [7]:
n = 5
non_hyb_stats = {i: {
        'correct': 0,
        'correct_parent': 0,
        'correct_length': 0,
        'correct_start': 0,
        'correct_end': 0
    } for i in range(n)}
non_hyb_stats['count'] = 0

hyb_stats = {
    'count': 0,
    'left': {
        'correct': 0,
        'correct_parent': 0,
        'correct_length': 0,
        'correct_start': 0,
        'correct_end': 0
    },
    'right': {
        'correct': 0,
        'correct_parent': 0,
        'correct_length': 0,
        'correct_start': 0,
        'correct_end': 0
    }
}


In [8]:
def hyb_calc(result, real_pep):
    left_corrparent = False
    left_corrstart = False
    left_corrend = False
    left_corrlen = False
    right_corrparent = False
    right_corrstart = False
    right_corrend = False
    right_corrlen = False
    iterrange = min(n, len(result))
    for i in range(iterrange):
        if result[i]['protein_name'] == real_pep['left_parent_name']:
            left_corrparent = True
            left_corrstart = left_corrstart or (result[i]['starting_position'] == real_pep['left_parent_starting_position'])
            left_corrend = left_corrend or (result[i]['ending_position'] == real_pep['left_parent_ending_position'])
            left_corrlen = left_corrlen or (result[i]['length'] == real_pep['left_parent_ending_position'] - real_pep['left_parent_starting_position'] + 1)
        if result[i]['protein_name'] == real_pep['right_parent_name']: 
            right_corrparent = True
            right_corrstart = right_corrstart or(result[i]['starting_position'] == real_pep['right_parent_starting_position'])
            right_corrend = right_corrend or (result[i]['ending_position'] == real_pep['right_parent_ending_position'])
            right_corrlen = right_corrlen or (result[i]['length'] == real_pep['right_parent_ending_position'] - real_pep['right_parent_starting_position'] + 1)
    
    hyb_stats['count'] += 1
    hyb_stats['left']['correct_parent'] += 1 if left_corrparent else 0
    hyb_stats['left']['correct_length'] += 1 if left_corrlen else 0
    hyb_stats['left']['correct_start'] += 1 if left_corrstart else 0
    hyb_stats['left']['correct_end'] += 1 if left_corrend else 0
    hyb_stats['left']['correct'] += 1 if (left_corrparent and left_corrlen and left_corrstart and left_corrend) else 0
    hyb_stats['right']['correct_parent'] += 1 if right_corrparent else 0
    hyb_stats['right']['correct_length'] += 1 if right_corrlen else 0
    hyb_stats['right']['correct_start'] += 1 if right_corrstart else 0
    hyb_stats['right']['correct_end'] += 1 if right_corrend else 0
    hyb_stats['right']['correct'] += 1 if (right_corrparent and right_corrlen and right_corrstart and right_corrend) else 0
    

In [9]:
def non_hyb_calc(result, real_pep):
    non_hyb_stats['count'] += 1
    iterrange = min(n, len(result))
    for i in range(iterrange):
        if result[i]['protein_name'] != real_pep['parent_name']:
            continue
        corrlen = len(result[i]['sequence']) == len(real_pep['sequence'])
        corrstart = result[i]['starting_position'] == real_pep['starting_position']
        corrend = result[i]['ending_position'] == real_pep['ending_position']
        
        non_hyb_stats[i]['correct_parent'] += 1 
        non_hyb_stats[i]['correct_length'] += 1 if corrlen else 0
        non_hyb_stats[i]['correct_start'] += 1 if corrstart else 0
        non_hyb_stats[i]['correct_end'] += 1 if corrend else 0
        non_hyb_stats[i]['correct'] += 1 if (corrend and corrlen and corrstart) else 0
        return


In [10]:
expfile = '../data/testing_output/experiment_info.json'
exp = json.load(open(expfile, 'r'))

scan_no_keyed_results = {x['scan_no']: x['alignments'] for _, x in summary.items()}
sorted_keys = [int(c) for c in exp['peptides'].keys()]

for k in sorted_keys:
    pep = exp['peptides'][str(k)]
    if 'hybrid' in pep['peptide_name'].lower():
        hyb_calc(scan_no_keyed_results[k], pep)
    else:
        non_hyb_calc(scan_no_keyed_results[k], pep)

In [11]:
percent = lambda a, b: (a* 100 // b)

printstat = lambda name, stat: '{}{}\n'.format(name, str(stat).rjust(60-len(name), '.'))

secbreak = ''.join(['=' for _ in range(60)])
headbreak = ''.join(['-' for _ in range(60)])
nhcount = non_hyb_stats['count']
topalign = non_hyb_stats[0]
otheralign = {}
for i in range(1, n):
    for stat in topalign.keys():
        if stat not in otheralign:
            otheralign[stat] = 0
        otheralign[stat] += non_hyb_stats[i][stat]

######################## NON HYBRID PRETTY PRINTING ############################

nonhybsum = 'NON HYBRID STATS\n' + headbreak + '\n'
nonhybsum += printstat('number of peptides', nhcount) 
nonhybsum += 'Top alignment\n\n'
nonhybsum += printstat('correct alignment', topalign['correct'])
nonhybsum += printstat('%', percent(topalign['correct'], nhcount)) 
nonhybsum += printstat('correct protein', topalign['correct_parent']) 
nonhybsum += printstat('%', percent(topalign['correct_parent'], nhcount))
nonhybsum += printstat('correct starting position', topalign['correct_start'])
nonhybsum += printstat('%', percent(topalign['correct_start'], nhcount))
nonhybsum += printstat('correct ending position', topalign['correct_end'])
nonhybsum += printstat('%', percent(topalign['correct_end'], nhcount))
nonhybsum += printstat('correct length', topalign['correct_length'])
nonhybsum += printstat('%', percent(topalign['correct_length'], nhcount))

nonhybsum += '\n2nd or 3rd alignment\n\n'
nonhybsum += printstat('number of peptides', nhcount) 
nonhybsum += printstat('correct alignment', otheralign['correct'])
nonhybsum += printstat('%', percent(otheralign['correct'], nhcount)) 
nonhybsum += printstat('correct protein', otheralign['correct_parent']) 
nonhybsum += printstat('%', percent(otheralign['correct_parent'], nhcount))
nonhybsum += printstat('correct starting position', otheralign['correct_start'])
nonhybsum += printstat('%', percent(otheralign['correct_start'], nhcount))
nonhybsum += printstat('correct ending position', otheralign['correct_end'])
nonhybsum += printstat('%', percent(otheralign['correct_end'], nhcount))
nonhybsum += printstat('correct length', otheralign['correct_length'])
nonhybsum += printstat('%', percent(otheralign['correct_length'], nhcount))
nonhybsum += '\n' + secbreak + '\n\n'

############################ HYBRID PRETYY PRINTING ##############################
hcount = hyb_stats['count']
lalign = hyb_stats['left']
ralign = hyb_stats['right']

hybsum = 'HYBRID STATS (if the top {} alignments contained either left or right it appears)\n'.format(n) + headbreak + '\n'
hybsum += printstat('number of peptides', hcount)
hybsum += '\nLeft \n\n'
hybsum += printstat('correct side alignment', lalign['correct'])
hybsum += printstat('%', percent(lalign['correct'], hcount))
hybsum += printstat('correct protein', lalign['correct_parent'])
hybsum += printstat('%', percent(lalign['correct_parent'], hcount))
hybsum += printstat('correct starting position', lalign['correct_start'])
hybsum += printstat('%', percent(lalign['correct_start'], hcount))
hybsum += printstat('correct ending position', lalign['correct_end'])
hybsum += printstat('%', percent(lalign['correct_end'], hcount))
hybsum += printstat('correct length', lalign['correct_length'])
hybsum += printstat('%', percent(lalign['correct_length'], hcount))

hybsum += '\nRight \n\n'
hybsum += printstat('correct side alignment', ralign['correct'])
hybsum += printstat('%', percent(ralign['correct'], hcount))
hybsum += printstat('correct protein', ralign['correct_parent'])
hybsum += printstat('%', percent(ralign['correct_parent'], hcount))
hybsum += printstat('correct starting position', ralign['correct_start'])
hybsum += printstat('%', percent(ralign['correct_start'], hcount))
hybsum += printstat('correct ending position', ralign['correct_end'])
hybsum += printstat('%', percent(ralign['correct_end'], hcount))
hybsum += printstat('correct length', ralign['correct_length'])
hybsum += printstat('%', percent(ralign['correct_length'], hcount))

print(nonhybsum + hybsum)

NON HYBRID STATS
------------------------------------------------------------
number of peptides......................................1000
Top alignment

correct alignment........................................982
%.........................................................98
correct protein..........................................988
%.........................................................98
correct starting position................................986
%.........................................................98
correct ending position..................................982
%.........................................................98
correct length...........................................988
%.........................................................98

2nd or 3rd alignment

number of peptides......................................1000
correct alignment..........................................5
%..........................................................0
correct protein...............

In [12]:
def printres(x):
    for a in x:
        print(a)
printres(scan_no_keyed_results[10])
print(exp['peptides']['10'])

{'starting_position': 98, 'ending_position': 109, 'length': 12, 'b_score': 1.0, 'y_score': 1.0, 'confidence': 100.0, 'protein_name': 'UCHL3_MOUSE', 'spectrum': [29.518008435000002, 58.028740435, 74.060040285, 86.060040435, 131.573511785, 142.602072435, 147.112804135, 171.112804435, 195.620993285, 211.131528435, 246.650085435, 252.642456785, 262.139747135, 284.196868435, 303.192117435, 309.663920285, 338.710674435, 345.182477285, 390.234710135, 395.73213793499997, 401.724509285, 421.255780435, 437.243066285, 452.75360143499995, 492.292894435, 504.277637135, 505.772522285, 516.801082935, 562.314554285, 574.314554435, 605.376958435, 618.320564135, 618.8565862849999, 638.362035935, 647.3673182849999, 676.414072435, 689.357678135, 790.456999435, 802.441742135, 873.478856135, 904.4999264349999, 1010.537768135, 1032.594889435, 1123.621832135, 1147.621832435, 1236.7058961349999, 1275.716795435, 1293.7273601349998], 'sequence': 'GLIHAIANNKDK'}
{'k': 12, 'sequence': 'GLIHAIANNKDK', 'starting_pos

In [6]:
times = [0.04854393005371094,
0.03188514709472656,
0.07952308654785156,
0.009988069534301758,
0.032685041427612305,
0.03956890106201172,
0.04300117492675781,
0.08851480484008789,
0.045289039611816406,
0.02736687660217285,
0.02323770523071289,
0.018954992294311523,
0.034613847732543945,
0.11969780921936035,
0.04238271713256836,
0.028224706649780273,
0.0736851692199707,
0.041391849517822266,
0.035143136978149414,
0.027536869049072266,
0.13624906539916992,
0.06007981300354004,
0.050293922424316406,
0.06956601142883301,
0.024131059646606445,
0.029160022735595703,
0.10336899757385254,
0.037492990493774414,
0.030508041381835938,
0.024775266647338867,
0.10678887367248535]

import statistics 

print(statistics.mean(times))

0.05044028835911905
